# Query the corrected OpenStreetMap database

Previously, the OpenStreetMap (OSM) dataset (XML file) relative to the city of Mestre Venezia was parsed in order to check the quality of the data. After this analysis, the dataset was saved as csv files after correcting for the data errors. The following .csv files were created:
- nodes.csv
- nodes_tags.csv
- ways.csv
- ways_nodes.csv
- ways_tags.csv

Subsequently, a new relational database called mestre.db was created with SQLite3 by importing these .csv file.

In this part of the project, the mestre.db database is queried to find some stats and information about the OSM dataset.

In [1]:
# import SQLIte3
import sqlite3

# connect to the database and create cursor for querying
conn = sqlite3.connect('mestre.db')
cursor=conn.cursor()

# function for query
def query_db(QUERY):
    cursor.execute(QUERY)
    results = cursor.fetchall()
    for r in results:
        print(r)

Check number of nodes and ways in the map:

In [2]:
print('Nodes: ')
QUERY = "SELECT COUNT(*) FROM Nodes"
query_db(QUERY)

print('Ways: ')
QUERY = "SELECT COUNT(*) FROM Ways"
query_db(QUERY)

Nodes: 
(528507,)
Ways: 
(81700,)


Check number of users that contributed to the OSM database.

In [3]:
QUERY = "SELECT way_node.user, COUNT(*) as total FROM (SELECT user FROM Ways UNION ALL SELECT user FROM Nodes) way_node GROUP BY way_node.user ORDER BY total DESC LIMIT 5;"
query_db(QUERY)

("b'DarkSwan_Import'", 153769)
("b'bellazambo'", 113694)
("b'Arlas'", 82596)
("b'Tizianos'", 50766)
("b'GatoSelvadego'", 39444)


Analyze restaurants in the OSM database.

In [4]:
print('Number of restaurants in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%restaurant%';"
query_db(QUERY)

print('\nMost popular cuisines:')
QUERY = "SELECT Nodes_tags.value, COUNT(*) AS tot FROM Nodes_tags WHERE Nodes_tags.id IN (SELECT DISTINCT(id) FROM Nodes_tags WHERE value LIKE '%restaurant%') AND Nodes_tags.key LIKE '%cuisine%' GROUP BY Nodes_tags.value ORDER BY tot DESC LIMIT 5;"
query_db(QUERY)

print('\nNumber of restaurants having phone number specified:')
QUERY = "SELECT COUNT(*) AS tot FROM Nodes_tags WHERE Nodes_tags.id IN (SELECT DISTINCT(id) FROM Nodes_tags WHERE value LIKE '%restaurant%') AND Nodes_tags.key LIKE '%phone%';"
query_db(QUERY)


print('\nTotal number of cusine specification:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE key LIKE '%cuisine%';"
query_db(QUERY)

Number of restaurants in Mestre Venezia:
(236,)

Most popular cuisines:
("b'italian'", 33)
("b'pizza'", 22)
("b'regional'", 12)
("b'italian;pizza'", 5)
("b'chinese'", 3)

Number of restaurants having phone number specified:
(43,)

Total number of cusine specification:
(129,)


Interestingly, there are 236 restaurants in the OSM database. However, only 129 restaurants (55%) specify the cusine. The most popular cusine is italian (33), followed by pizza (22). By considering that pizza is a particular italian speciality, 72 restaurants on the 129 (therefore 56%) with specified cuisine serve italian specialities.

However, only 43 restaurants have phone number specified. Therefore, another resource must be employed to check the phone number out and book a table.

In addition to the 236 restaurants, there are 36 pubs, 179 bars and 77 cafes in the city, as shown by the the following query: 

In [5]:
print('Number of pubs in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%pub%';"
query_db(QUERY)

print('Number of bars in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%bar%';"
query_db(QUERY)

print('Number of cafes in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%cafe%';"
query_db(QUERY)

Number of pubs in Mestre Venezia:
(36,)
Number of bars in Mestre Venezia:
(179,)
Number of cafes in Mestre Venezia:
(77,)


Check public transportation stops.

In [6]:
print('Number of stops for public transportation in Mestre Venezia:')
QUERY = "SELECT value,COUNT(*) AS tot FROM Nodes_tags WHERE value LIKE '%bus_stop%' OR value LIKE '%tram_stop%' GROUP BY value ORDER BY tot DESC ;"
query_db(QUERY)


Number of stops for public transportation in Mestre Venezia:
("b'bus_stop'", 499)
("b'tram_stop'", 70)
("b'bus stop only, private lane'", 1)
("b'bus stop to PLUS Camping Jolly'", 1)


The city of Mestre has about 571 stops for public transportation. Among them, 87% of the stops are for busses. In fact, the tram was built in 2015 and it consists of only two lines. On the contrary, bus public transportation has about 50 lines, which results in much more stops than tram.

Check number of hotels.

In [7]:
print('Number of Hotels in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%hotel%';"
query_db(QUERY)

Number of Hotels in Mestre Venezia:
(291,)


Check number of supermarkets in the city.

In [8]:
print('Number of supermarket in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%supermarket%';"
query_db(QUERY)

print('\nMost numerous supermarkets:')
QUERY = "SELECT Nodes_tags.value, COUNT(*) AS tot FROM Nodes_tags WHERE Nodes_tags.id IN (SELECT DISTINCT(id) FROM Nodes_tags WHERE value LIKE '%supermarket%') AND Nodes_tags.key LIKE '%name%' GROUP BY Nodes_tags.value ORDER BY tot DESC LIMIT 5;"
query_db(QUERY)

Number of supermarket in Mestre Venezia:
(46,)

Most numerous supermarkets:
("b'Prix'", 5)
("b'Conad City'", 4)
("b'Coop'", 4)
("b'Cadoro'", 3)
("b'Lidl'", 3)


Check number of doctors in the city. Interestingly, the OSM database for the city of Mestre Venezia has 1 doctor. This field should be improved in the future.

In [9]:
print('Number of doctors in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%doctor%';"
query_db(QUERY)


Number of doctors in Mestre Venezia:
(1,)


Check pedestrian crossing and traffic signals.

In [10]:
print('Number of traffic signals in Mestre Venezia:')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%traffic_signal%';"
query_db(QUERY)

print('\nNumber of street crossing area in Mestre Venezia: ')
QUERY = "SELECT COUNT(*) FROM Nodes_tags WHERE value LIKE '%crossing%';"
query_db(QUERY)

Number of traffic signals in Mestre Venezia:
(124,)

Number of street crossing area in Mestre Venezia: 
(1355,)


## Conclusions

In this project, the Mestre Venezia OpenStreetMap dataset (XML file, 113 MB) was analyzed, cleaned and saved into a structured database. A lot of errors (mainly typos) were found. As regards to the phone number field, most of the correctons were made in order to have numbers in the same format.

The databased was analyzed by querying few fields. We found that the info about public transportation and restaurants is quite complete, since most of these objects are indicated as nodes in the map. As regard to the restaurant, information are incomplete, since only few restaurants have info on cuisine and phone number. Therefore, the OSM datasef of the city of Mestre Venezia should be improved.